In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

try:
    path_to_db = r'C:\Users\ihatewindows\Downloads\project2\telecomm_csi.db'
except:
    path_to_db ="/datasets/telecomm_csi.db"
    
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [2]:
#CODE REVIEW
import os

In [3]:
#CODE REVIEW
if os.path.exists(r'C:\Users\ihatewindows\Downloads\project2\telecomm_csi.db'):
    path_to_db = r'C:\Users\ihatewindows\Downloads\project2\telecomm_csi.db'
else:
    path_to_db ="/datasets/telecomm_csi.db"
    
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [4]:
engine.url

sqlite:////datasets/telecomm_csi.db

In [5]:
query_l = """
SELECT * FROM location;
"""

In [6]:
df_l = pd.read_sql(query_l, engine)
df_l.head(3)

,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия


In [7]:
query_u = """
SELECT *
FROM user;
"""

In [8]:
df_u = pd.read_sql(query_u, engine)
df_u.head(3)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10


In [9]:
query_ag = """
SELECT * FROM age_segment;
"""

In [10]:
df_ag = pd.read_sql(query_ag, engine)
df_ag.head(3)

,age_gr_id,bucket_min,bucket_max,title
0,1,NaN,15.0,01 до 16
1,2,16.0,24.0,02 16-24
2,3,25.0,34.0,03 25-34


In [11]:
query_ts = """
SELECT * FROM traffic_segment;
"""

In [12]:
df_ts = pd.read_sql(query_ts, engine)
df_ts.head(3)

,tr_gr_id,bucket_min,bucket_max,title
0,1,0.00,0.00,01 0
1,2,0.00,0.01,01 0-0.01
2,3,0.01,0.10,02 0.01-0.1


In [13]:
query_ls = """
SELECT * FROM lifetime_segment;
"""

In [14]:
df_ls = pd.read_sql(query_ls, engine)
df_ls.head(3)

,lt_gr_id,bucket_min,bucket_max,title
0,1,1.0,1.0,01 1
1,2,2.0,2.0,02 2
2,3,3.0,3.0,03 3


In [15]:
query = """
SELECT user.user_id, user.lt_day,
CASE
    WHEN lt_day > 365
        THEN 'old_client'
    ELSE 'new_client'
END AS is_new,
       user.age, 
CASE
    WHEN gender_segment = 0 
        THEN 'male'
    WHEN gender_segment = 1
        THEN 'female'
    ELSE 'non-binary'
END AS gender_segment,
       user.os_name,
       user.cpe_type_name, location.country, location.city, age_segment.title as age_segment,
       traffic_segment.title as traffic_segment, lifetime_segment.title as lifetime_segment, 
CASE 
    WHEN nps_score >= 9
        THEN 'promoters'
    WHEN nps_score >= 7 AND nps_score < 9
        THEN 'passives'
    ELSE 'detractors'
END AS nps_group, 
    user.nps_score
FROM user 
    INNER JOIN location ON user.location_id = location.location_id
    INNER JOIN age_segment ON age_segment.age_gr_id = user.age_gr_id
    INNER JOIN traffic_segment ON traffic_segment.tr_gr_id = user.tr_gr_id
    INNER JOIN lifetime_segment ON lifetime_segment.lt_gr_id = user.lt_gr_id
;
"""

In [16]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_group,nps_score
0,A001A2,2320,old_client,45.0,female,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,promoters,10
1,A001WF,2344,old_client,53.0,male,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,promoters,10
2,A003Q7,467,old_client,57.0,male,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,promoters,10


In [17]:
df.to_csv(r'C:\Users\ihatewindows\Downloads\project2\telecomm_csi_tableau.csv', index=False)

### Укажите ссылку на дашборд на сайте Tableau Public:

https://public.tableau.com/views/project_2_16640174911080/Sheet10?:language=en-US&:display_count=n&:origin=viz_share_link

### Укажите ссылку на pdf-файл с презентацией:

https://drive.google.com/file/d/15ZXyeCLDqCIwCSWUp5SfvX1OkvwVEOYk/view?usp=sharing